* **Image Reading**
* **Generating Super pixels**
* **Generating Perturbations**
* **Making predictions**
* **Compute distance and weights**
* **Fitting a Linear regression**
* **Generating the mask**

In [5]:
import tensorflow
import numpy as np
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import skimage.io 
import skimage.segmentation
import copy
import sklearn
import sklearn.metrics
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import urllib
warnings.filterwarnings("ignore")
print('Notebook running: keras ', tensorflow.keras.__version__)
np.random.seed(222)

Notebook running: keras  2.7.0


In [ ]:
def readImage(Image_Path):
    image = cv2.imread(Image_Path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = skimage.transform.resize(image, (224 ,224)) 
    return image

def draw_Img(img, title = "Image"):
    plt.imshow(img)
    plt.axis('off')
    plt.style.use('seaborn')
    plt.title(title+str(img.shape))
    skimage.io.imshow(Image) 
    plt.show()

def Generate_Superpixels(Image, pixel_dist):
    superpixels = skimage.segmentation.quickshift(Image, kernel_size = 3, max_dist = pixel_dist, ratio = 0.2)
    num_superpixels = np.unique(superpixels).shape[0]
    print(f"Number of Super pixels generated: {num_superpixels}")
    return num_superpixels, superpixels

def Generate_Perturbations(num_perturbations):
    perturbations = np.random.binomial(1, 0.5, size=(num_perturbations, num_superpixels))
    # each row represnt each perturbation 
    # each column represent the super pixels
    print(f"Changes in the superpixels in each perturbations are: {len(perturbations[0])}")
    print(f"Number of perturbations: {len(perturbations)}")
    return perturbations

def perturb_Image(img, perturbation, segments):
    active_pixels = np.where(perturbation == 1)[0]
    mask = np.zeros(segments.shape)
    for active in active_pixels:
        mask[segments == active] = 1 
    perturbed_image = copy.deepcopy(img)
    perturbed_image = perturbed_image*mask[:,:,np.newaxis]
    return perturbed_image

def Compute_Distances(num_superpixels, perturbations):
    original_image = np.ones(num_superpixels)[np.newaxis,:] #Perturbation with all superpixels enabled 
    distances = sklearn.metrics.pairwise_distances(perturbations,original_image, metric='cosine').ravel()
    distances.shape
    return distances


def Compute_weights(distances, kernel_width):
    weights = np.sqrt(np.exp(-(distances**2)/kernel_width**2)) #Kernel function
    return weights

def fit_weights(perturbations, predictions, weights):
    simpler_model = LinearRegression()
    simpler_model.fit(X=perturbations, y=predictions[:,:,0], sample_weight=weights)
    coeff = simpler_model.coef_[0]
    return coeff

def mask_image(Image, num_superpixels, super_pixels, num_top_features, coeff):
    top_features = np.argsort(coeff)[-num_top_features:] 
    mask = np.zeros(num_superpixels) 
    mask[top_features]= True #Activate top superpixels
    skimage.io.imshow(perturb_Image(Image,mask,super_pixels))

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

#### Image Reading

In [ ]:
# Image_Path =  r"C:\Users\Teja Nikhil\OneDrive\Desktop\COVID-19_Radiography_Dataset\Testing\COVID-22.png"
Image_Path = r"C:\Users\Teja Nikhil\OneDrive\Desktop\Test1.png"
Image = readImage(Image_Path)
plt.imshow(Image)

#### Generating Superpixels

In [ ]:
# draw_Img(Image)
num_superpixels , super_pixels = Generate_Superpixels(Image, 200)
skimage.io.imshow(skimage.segmentation.mark_boundaries(Image, super_pixels))

#### Generating Perturbations

In [ ]:
perturbations = Generate_Perturbations(150)
skimage.io.imshow(perturb_Image(Image, perturbations[0],super_pixels))

In [ ]:
Perturbated_Df = pd.DataFrame(perturbations.flatten())
Perturbated_Df.shape
predictions = []

#### Making Predictions

In [ ]:
for pert in perturbations:
    perturbed_img = perturb_Image(Image,pert,super_pixels)
    pred = model.predict(perturbed_img[np.newaxis,:,:,:])
    predictions.append(pred)

In [ ]:
predictions = np.array(predictions)

#### Compute distances and weights

In [ ]:
distances = Compute_Distances(num_superpixels, perturbations)

In [ ]:
weights = Compute_weights(distances, 0.25)

#### Linear regression using predictions and weights

In [ ]:
coeffs = fit_weights(perturbations, predictions, weights)

#### Plotting the masked image

In [ ]:
mask_image(Image, num_superpixels, super_pixels, 10, coeffs)